In [1]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from __future__ import annotations
from pathlib import Path
from datetime import datetime
import numpy as np

In [3]:
from Chemprompt.embeddings.no_description_embedding import LLMModel
from Chemprompt.data.data_loader import DataLoader
from Chemprompt.models.sklearn_model import ScikitLearnModel

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/HDD1/bbq9088/miniconda3/envs/ChEmPrompt1027/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'
Skipped loading some PyTorch models, missing a dependency. No module named 'tensorflow'


In [4]:
DATASETS = ["FreeSolv","ESOL","Lipo"] # dataset

In [5]:
MODEL_LIST = [ # {"repo": "model_repo", "name" : "model_name"}
    # {"repo": "meta-llama", "name": "Meta-Llama-3.1-8B"},
    {"repo": "CohereLabs", "name": "aya-expanse-8b"},
]

In [6]:
RESULT_ROOT = Path("./result/no_description").resolve()
RESULT_ROOT.mkdir(parents=True, exist_ok=True)

In [7]:
def log(msg: str):
    print(f"[{datetime.now():%Y-%m-%d %H:%M:%S}] {msg}")

In [8]:
def train_and_predict(x, embeds, y, save_dir: Path):
    model = ScikitLearnModel(
        model_type="regression",
        save_dir=str(save_dir),
        max_iter=5000
    )
    model.fit_and_evaluate_fold(x, embeds, y)
    # np.save(save_dir / "embeddings.npy", embeds)

In [9]:
def run_all_descriptors(dataset: str, model_info: dict):
    repo, name = model_info["repo"], model_info["name"]
    model_repo = f"{repo}/{name}"

    # data load
    loader = DataLoader()
    smiles, y = loader.load_dataset(dataset)

    # embedding generation
    # llm = LLMModel(model_repo, dtype="half", device = "cuda:0") # float 16
    llm = LLMModel(model_repo, dtype="full", device = "cuda:0") # float 32
    log(f"\n[INFO] Model: {model_repo} | Dataset: {dataset}")
    embeddings = llm.get_embeddings(smiles)

    # save result
    save_dir = RESULT_ROOT / dataset / name
    save_dir.mkdir(parents=True, exist_ok=True)
    log(f"Training & saving to {save_dir}")
    train_and_predict(smiles, embeddings, y, save_dir)

In [10]:
if __name__ == "__main__":
    for dataset in DATASETS:
        log(f"\n=== Dataset: {dataset} ===")
        for model in MODEL_LIST:
            run_all_descriptors(dataset, model)

[2025-10-29 05:56:09] 
=== Dataset: FreeSolv ===
(642, 2)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


[2025-10-29 05:56:14] 
[INFO] Model: meta-llama/Meta-Llama-3.1-8B | Dataset: FreeSolv


Processing SMILES with Properties: 100%|████████████████████████████████████████████████████| 642/642 [00:32<00:00, 19.74it/s]


[2025-10-29 05:56:46] Training & saving to /HDD1/bbq9088/ChEmPrompt_Lab/5.1 Prompt with Descriptors is better than Ones with No Descriptor/result/no_description/FreeSolv/Meta-Llama-3.1-8B
Predictions saved to /HDD1/bbq9088/ChEmPrompt_Lab/5.1 Prompt with Descriptors is better than Ones with No Descriptor/result/no_description/FreeSolv/Meta-Llama-3.1-8B/regression_Predictions.csv
Combined results saved to /HDD1/bbq9088/ChEmPrompt_Lab/5.1 Prompt with Descriptors is better than Ones with No Descriptor/result/no_description/FreeSolv/Meta-Llama-3.1-8B/regression_CombinedResults.csv
(642, 2)


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]


KeyboardInterrupt

